In [1]:
#!/usr/local/bin/python
# -*- coding: utf-8 -*-
### export PYTHONIOENCODING=utf-8  # at cmd of linux
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.initializers import Constant
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers
from keras import backend as K
from tensorflow.keras.callbacks import *
from tensorflow.keras import utils
import tensorflow as tf

import torch

Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import pickle
import time
import re
import nltk
import sys
import html
import xml.sax.saxutils as saxutils
from html.parser import HTMLParser
from io import StringIO
import random
import operator
from transformers import BertTokenizer, BertModel
from transformers import DistilBertModel,DistilBertTokenizer

from scipy.spatial.distance import cosine

from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
Stem=stemmer.stem

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
lemm=wordnet_lemmatizer.lemmatize

In [3]:
from gensim.models.fasttext import load_facebook_model

import fasttext.util
#fasttext.util.download_model('en', if_exists='ignore')  # English

In [4]:

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model_bert = DistilBertModel.from_pretrained('distilbert-base-uncased',
                                       output_hidden_states = True, # Whether the model returns all hidden-states.
                                       )  
'''
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = BertModel.from_pretrained('bert-base-uncased',
                                       output_hidden_states = True, # Whether the model returns all hidden-states.
                                       )
'''
print()

In [5]:
#print(len(cachedStopWords))
#print(len(cachedStopWords))
#print(type(cachedStopWords))

nltk_stopwords=set(cachedStopWords)

english_alghabet=['b','c','e','f','g','h','j','k','l','n','p','q','r','u','v','w','x','z']

numbers_remove=['one','two','three','four','five','six','seven','eight','nine','ten','tens','twenty',
                'fourty','fifty','sixty','seventy','eighty','ninety','hundred','hundreds','million','billion','trillion',
                'millions','thousand','thousands','second','third','forth','tenth','billions','trillions'] 

miscellaneous_remove=['absolutely', 'actually', 'adieu', 'ain', "ain't", 'aint', 'almost',
                       'awesome','awfully','amazing','interesting',
                       'alright','alrighty', 'amoungst', 'anybody', 'anymore', 'anyways', 'apart', 'apparently', 'anytime',
                       'appropriate',  'approximately', 'arent', 'behold', 'better', 'bravo','briefly','bad','best','brilliant',
                       'bye', 'cant', 'certainly', 'chrissakes', 'clearly', 'completely',
                       'congrat', 'congrats','congratulation', 'congratulations', 'consequently', 'cool', 'couldnt',
                       'darnit', 'de','dear', 'definitely','disappointing', 'didn', 'doesn', 'don', 'downwards',
                       'disgusting','dude','down','eg',"e.g.",'i.e.',
                       'encore','entirely', 'especially', 'et', 'etc', 'everybody', 'ex', 'exactly', 'excellent',
                       'fantastic','far', 'farewell','funny',
                       'felicitation', 'felicitations','finally', 'fully','furthermore', 'gadzooks', 
                       'good', 'goodby','goodness', 'gracious', 'great', 
                       'greetings', 'hallo', 'hardly', 'hasnt', 'haven', 'hello', 'here','hi', 'hither','higher','hopefully',
                       'here','there','including',
                       'howbeit', 'ie', 'immediately', 'inasmuch', 'inner', 'insofar', 'instead', 'inward', 'important',
                       'indeed','just', "it'd", "it'll", 'inside','kertyschoo', 'kg', 'km', 'lackaday', 
                       'largely', 'lately', 'later','lovely','large','big','small',
                       'lest', 'let', 'lets', 'likely', 'little', 'ltd', 'lower','magnificent', 'mainly', 'marvelous',
                       'myself','yourself','yourselves','himself','herself','hisself','ourselves','themsleves',
                       'maybe', 'meantime', 'merely', 'minus', 'near', 'nearly', 'necessary', 'never', 
                       'non', 'normally', 'obviously', 'ok', 'okay', 'ones', 'outside', 'over','other','others','only',
                       'overall', 'particular', 'particularly', 'please', 'plus', 'poorly', 'possible','up',
                       'possibly', 'potentially', 'predominantly', 'presumably', 'previously','primarily', 'probably',
                       'promising',
                       'promptly', 'readily', 'really', 'reasonably', 'recent', 'recently', 'ref',
                       'refs', 'regardless', 'related', 'relatively', 'respectively', 'resulting', 'right', 'sec', 
                       'secondly','self', 'selves', 'seriously', 'shall', 'shucks','somebody', 'somethan','sorry',
                       'somewhat', 'soon', 'late' , 'sorry', 'stupid', 'sub', 'substantially', 'successfully', 'sufficiently',
                       'useful',
                       'super', 'sure', "t's", 'th', 'thank', 'thanks', 'thanx', "that've", 'thats', 'there', "there'll",
                       "there've", 'thered', 'thereof', 'therere', 'theres', 'thereto', 'theyd', 'theyre', 'thorough',
                       'then','thankfully','too','today','yesterday','tomorrow','night',"morning",'afternoon','noon','tonight',
                       'evening','day','everyday', 'everynight','todays','nights','mornings','noons','afternoons','days',
                       'evenings','week','month','year',
                       'thoroughly', 'tnx', 'too','truly', 'twice', 'undoubtedly','unfortunately', 'unlike','unlikely',
                       'unto',  'usually', 'vs', 'welcome', 'well', 'went', 'werent', 'what', 'whatever', 'wheres', 'widely',
                       'wonderful', 'wont', 'wouldnt', 'wrong', 'worst','worse','www', 'yes', 'youd', 'youre', 'yummy', 
                       'zoinks','shit','literally','literal','pleasure','effective','fabulous','delighted',
                       'saturday','sunday','monday','tuesday','wednesday','thursday', 'friday','past','future','suitable',
                       'much','many','less','least','few','lots','lot','fewer','fewset','therefore','pm',
                       'afaik', 'br', 'idk','smh','qotd', 'ftw','bfn','yw', 'icymi','fomo','smdh', 'b4','imho',
                       'urdddd','fab' ,'delightful','absolute','pleasure','huge','latest','nowadays',
                       'january','february','april','june','july','august','september','october',
                       'november','december', 'autumn' ,'spring','winter','summer',
                       'mr','madam','sir','mrs','easy', 'difficult',
                       'weekend','south','north','west','east','asia','africa','europe','america','totally',
                       'come', 'comes', 'coming', 'came', 'seems', 'gives', 'gave', 'makes', 'made', 'keeps', 'kept', 
                       'calls', 'called', 'says', 'saying', 'said', 'goes', 'went', 'gone', 'got', 'saw', 'seen', 'shows',
                       'shown', 'took', 'taken', 'uses', 'moved', 'moves', 'puts',
                       'using','seem','give','make','keep','call','say','go','get','see','seems','seeming',
                       'seemed','show','take','made','used','move','become','became','becoming','becomes','put','use',
                       'find', 'finds', 'finding','aka',
                       'lol' , 'brb', 'lmk', 'ama', 'tbh', 'irl', "tl;dr", 'fml', 'bfn' ,' br', 'ht', "hth",'j/k', 'lmao' ] #cool

interjection_remove=['aaaahh', 'aaah', 'aaargh', 'aaay', 'aagh', 'aah',
                   'aargh', 'achoo', 'adios', 'ah', 'aha', 'ahem', 'ahh', 'ahhh',
                   'ahoy', 'alas', 'allo', 'amen', 'areet', 'argh', 'arrggh',
                   'arrividerci', 'asap', 'attaboy', 'avaunt', 'aw', 'aw', 'aww',
                   'awww', 'ay', 'ay', 'aye', 'ayeaugh', 'bada', 'badum', 'bah',
                   'bahaha', 'bam', 'bazinga', 'behold', 'bingce', 'bingo', 'blah',
                   'blech', 'bleh', 'blimey', 'bonjour', 'boo', 'booh', 'boohoo',
                   'booyah', 'bravo', 'brr', 'brrrr', 'btw', 'bwahaha', 'capeesh',
                   'capisce', 'cheerio', 'cheers', 'ciao', 'cor', 'cowabunga',
                   'crikey', 'cripes', 'da', 'dabba', 'dah', 'dammit', 'damn', 'dang',
                   'darn', 'de', 'dee', 'di', 'dizamn', 'doh', 'doo', 'drat', 'duh',
                   'dum', 'eeeek', 'eek', 'eep', 'egad', 'egads', 'eh', 'ehem', 'em',
                   'er', 'eureka', 'eww', 'ewww', 'eyh', 'fiddledeedee', 'fie',
                   'fore', 'foul', 'fuff', 'gah', 'gak', 'gee', 'geez', 'gesundheit',
                   'giddyap', 'golly', 'gosh', 'grr', 'grrrr', 'ha', 'hah', 'haha',
                   'hahaha', 'hallelujah', 'halloa', 'harrumph', 'harumph', 'haw',
                   'heck', 'heck', 'heeey', 'heh', 'hehe', 'hey', 'hhh', 'hic', 'hm',
                   'hmm', 'hmmm', 'hmmmm', 'hmmph', 'hmpf', 'ho', 'hola', 'hoo',
                   'hooray', 'howdy', 'hrmm', 'hrmph', 'hrmph', 'hrrmph', 'hu', 'huh',
                   'hullo', 'humph', 'hurrah', 'huzza', 'huzzah', 'ich', 'ick',
                   'ixnay', 'jeepers', 'jeez', 'kaboom', 'kapow', 'kerwham', 'la',
                   'lala', 'lo', 'lordy', 'meh', 'mhm', 'ml', 'mm', 'mmh', 'mmhm',
                   'mmm', 'muahaha', 'mwah', 'mwahaha', 'na','nay','nah', 'nanu', 'nooo', 'nope',
                   'nuh', 'oh', 'ohh', 'oho', 'oi', 'okeydoke', 'om', 'oof', 'ooh',
                   'oomph', 'oooh', 'ooooh', 'oops', 'ouch', 'ow', 'oww', 'oy',
                   'oyez', 'oyh', 'pew', 'pff', 'pffh', 'pfft', 'phew', 'phut',
                   'phweep', 'phwoar', 'phwoarr', 'poof', 'poogh', 'prethee',
                   'prithee', 'prosit', 'pssh', 'psst', 'queep', 'roger', 'salaam',
                   'salam', 'sheesh', 'shh', 'shhh', 'shitfire', 'shoo', 'shoop',
                   'shush', 'sigh', 'sssh', 'strewth', 'ta', 'tarnations', 'tchah',
                   'teehee', 'tish', 'touché', 'tsk', 'tss', 'tut', 'uggh', 'ugh',
                   'uh', 'uhh', 'uhm', 'um', 'umm', 'ummm', 'umph', 'unh', 'upadaisy',
                   'upsadaisy', 'ur', 'urgh', 'vay', 'vayf', 'viva', 'voila', 'waa',
                   'waaaaah', 'waah', 'wah', 'wahey', 'wassup', 'weee', 'welp',
                   'wham', 'whamo', 'whee', 'whew', 'whizz', 'whoa',
                   'whoo', 'whoopee','whoop', 'whoops', 'whoopsy', 'whoosh', 'woah', 'woo',
                   'woohoo', 'wotcha', 'wotcher', 'wow', 'wowsers', 'wowsers',
                   'wuzzup', 'wuzzup', 'wuzzup', 'ya', 'yabba', 'yada', 'yadda',
                   'yak', 'yarooh', 'yay', 'yea', 'yeah', 'yech', 'yee', 'yeeeeaah',
                   'yeehaw', 'yeow', 'yes', 'yessiree', 'yew', 'yikes', 'yippee',
                   'yo', 'yoo', 'yoohoo', 'yow', 'yowza', 'yuck', 'yuh', 'zing',
                   'zoiks', 'zomfg', 'zomg', 'zounds', 'zut']
             
import spacy
sp = spacy.load('en_core_web_sm')
spacy_stopwords = sp.Defaults.stop_words
type(spacy_stopwords)
#spacy_exclude=['using','name','seem','give','make','keep','call','say','go','get','see','seems','seeming',
#               'seemed','show','take','made','used','move','become','became','becoming','becomes','put','use']# serious

from stop_words import get_stop_words
stop_words = get_stop_words('en')
stop_words1 = get_stop_words('english')
#print(type(stop_words1))
#print()
#print(stop_words1)
lib_stopwords=set(stop_words1)

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
#print(type(ENGLISH_STOP_WORDS))
#print()
#print(set(ENGLISH_STOP_WORDS))
#sklearn_exclude=['find','get','found','go','see','seem','seems','give','seemed','take','keep','show','put','made'] # system  cry
sklearn_stopwords=set(ENGLISH_STOP_WORDS)

#spacy_stopwords.difference_update(set(spacy_exclude))
#sklearn_stopwords.difference_update(set(sklearn_stopwords))
#for removing "just" one item, use "remove"
temp_1=set([])
#temp_1.update(nltk_stopwords)
#temp_1.update(lib_stopwords)
#temp_1.update(sklearn_stopwords)
#temp_1.update(spacy_stopwords)
#temp_1.update(set(english_alghabet))
#temp_1.update(set(numbers_remove)) 
#temp_1.update(set(miscellaneous_remove))
#temp_1.update(set(interjection_remove))
#temp_1.update(['rt','be','will','was','were','is','am','are','have','has','had','do','does','done'])
#temp_1.update(['rt'])
cachedStopWords=temp_1
#len(cachedStopWords)

In [6]:
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [7]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

#print(lemmatizer.lemmatize("I am loving it")) #I am loving it
#print(lemmatizer.lemmatize("loving")) #loving
#print(lemmatizer.lemmatize("loving", "v")) #love
#print(lemmatize_sentence("I am loving it")) #I be love it

In [8]:
def cleaning (text):
    
    #order of lines is important
    
    text=strip_tags(text)
    #text=html.unescape(text)   # stripping or converting html entities 
    #text=saxutils.unescape(text) 
    
    #convertings words that their lower and uper cases are different
    text=re.sub(" US | U\.S\. ", ' USA ', text) # before lower
    
    #converting
    text = re.sub("“|”", ''' " ''', text)  #before next lines
    text = re.sub("’|′|‘|`", " ' ", text)  #before next lines
    
    #removing tabs and lines
    text=re.sub('\t|\n', ' ', text)
    
    #converting lower_case
    text = text.lower() 
    
    #converting
#    text=re.sub('\$|£|€|¥|dollar|dollars|yen|yens|euros', ' money ', text)   # not euro 
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    
    #removing emoji
    text = emoji_pattern.sub(r' ', text) 

    #removing emojis and non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+|,Ä¶',' ', text)  

    #removeing http and https (URL)
    text = re.sub(r'(http://|https://)\S+', '', text)
    
    #removing www (URL)
    text=re.sub(r'www\.\S+', '', text)
    
    #removing targets
    text=re.sub('( |^)@\S+', '', text) 

    '''
    #removing common expressions
    text=re.sub("looking forward to|look forward to|make sure|kidding me|\
                |in my opinion|by the way,|as soon as possible|shaking my head|i don't know|I do not know|\
                |in real life|quote of the day|as far as i know|shake my head|\
                |to be honest|in other words|let me know|just kidding|hope that helps|hat tip|\
                |just like that|happy birthday|never mind|well-done|\
                |in my humble opinion|happy new year|you're welcome|you are welcome| \
                |it doesn't matter|it does not matter|i think|i wonder|do you think", ' ', text)  
    '''
    
    #convertings
    text=re.sub("can't", 'cannot', text) # before other n't 
    text=re.sub("can not ", 'cannot ', text)  
    text=re.sub("'ve",' have', text)
    text=re.sub("n't",' not', text)
    text=re.sub("'ll",' will', text)
#    text=re.sub("'d",' would', text)
    text=re.sub("'re",' are', text)
    text=re.sub("i'm",'i am', text)
    text=re.sub("&",' and ', text)
    text=re.sub(" w/ ",' with ', text)
    text=re.sub(" w/i | w/in ",' within ', text)
    text=re.sub(" w/o ",' without ', text)
    text=re.sub(" c/o ",' care of ', text)
    text=re.sub(" h/t ",' hat tip ', text)
    text=re.sub(" b/c ",' because ', text)
#    text=re.sub("=",' equals to ', text)
    text=re.sub("=",' = ', text)
#    text=re.sub("\+",' plus ', text)
    text=re.sub("\+",' + ', text)
    text=re.sub("united states",'usa', text)
    text=re.sub("united kingdom",'uk', text)
    text=re.sub(" the us ",' usa ', text)
    text=re.sub("start-up|start_up",'startup', text)
    text=re.sub("u\.s\.a", 'usa', text)  #try text=re.sub("u.s.a", 'usa', text) with text=substantially 
    #text=re.sub("aka", 'also known as', text)     
    text=re.sub("'"," ' ", text)     
    
    text= re.sub("(\?)+", '? ',text)     
    text= re.sub("(!)+", '! ',text)     
    text= re.sub("(\.\.)+", ' ',text)   

#    text = "".join(lemmatize_sentence(text))
    
    #removing some special charachter  
#    text= re.sub("[\"\+\-\|\*\?\(\)\/\\\^\[\]``<>\.{}`′’‘'_;•«»,@:~!\=%&]+", ' ',text) 
#    text= re.sub("[\"\“\”\+\-\|\*\?\(\)\/\\\^\[\]\.{}_`′’‘';•«,@:~!\=%&]+", ' ',text) 
    
    #removing hashtag
#    text=re.sub('#', ' ', text) 
    
    #removing numbers not attached to alphabets
    '''
    text=re.sub("(^)(\d+)?(\.)?(\d+)? ",' ',text)   #removing numer at the beginning
    text=re.sub("(\s)[0-9]?(\.)?(\d+) ",' ',text) #py6 and py9
    text= re.sub(" (\.)(\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) (\d+) (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) ", ' ',text)
    text= re.sub(" (\d+)$", ' ',text)
    '''
    #text=re.sub("\S+(\d+) ",' ',text) # alphabet+digit (attached)
    #text=re.sub(" (\d+)\S+",' ',text) # digit+alphabet (attached)
    #text=re.sub(" \S+(\d+)\S+ ",' ',text) # alphabet+digit+alphabet (attached)
    #text=re.sub("(\d+)",' ',text)  #removing any number anywhere but keeps \. for decimal numbers

    #removing space
    text=re.sub('\s+',' ',text)    
    
    text=re.sub('(^)rt ','',text)    # if we do not want to remove stopwords

#    text= nltk.word_tokenize(text) # necessary for removing stopwords
    #text= text.split() #sometimes

    #removing_stopwords 
    #text_without_sw = [word.lower() for word in text if word.lower() not in stopwords.words()] #very slow
#    text = [word for word in text if word not in cachedStopWords]

    #lemmatization
    #text= [ lemm(word, pos="v") for word in text]
    #text= [ lemm(word, pos="n") for word in text]
    #text= [ lemm(word, pos="a") for word in text]
    
    #stemming 
    #text = [Stem(word) for word in text]
    
#    text=' '.join(text)
#    text=re.sub("''",'''"''', text)    #since nltk.tokenize converts second " to ''"
#    text=re.sub("``",'''"''', text)   # since nltk.tokenize converts first " to " ``
    
    return text

In [9]:
'''
import psycopg2
con = psycopg2.connect(database="postgres", user="postgres", password="Jafarsql", host="localhost", port="5432")
print("Database opened successfully")

cur = con.cursor()
##cur.execute("SELECT user_id, tweet from ent_2019_100K limit 100000 ")
cur.execute("SELECT user_id, tweet from ent_2019_1000k ")
rows_ent = cur.fetchall()
con.close()

print(len(rows_ent))
'''
pass

In [10]:

#df_ent = pd.read_csv('/archives1/Datasets/TweetsWorld/ent_tweets_world.csv', delimiter='\t', na_values=".",error_bad_lines=False)#,warn_bad_lines=False)
df_ent = pd.read_csv('ent_tweets_world.csv', delimiter='\t', na_values=".",error_bad_lines=False)#,warn_bad_lines=False)
print(df_ent.shape)
print(df_ent.columns)
#print(df_ent.head()) # Preview the first 5 lines of the loaded data 

rows_ent=list(df_ent[['user_id', 'tweet','tweet_created_at']].itertuples(index=False, name=None)) #rows_ent0
#rows_ent= list(zip(df_ent.user_id, df_ent.tweet))
#rows_ent=df_ent[['user_id','tweet']].apply(tuple, axis=1) 
del df_ent

print("Number of tweets in ent:",len(rows_ent))  #rows_ent0
print('Memory size of ent:',sys.getsizeof(rows_ent)) #rows_ent0

pass

C:\Users\jafar\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


(47604376, 4)
Index(['user_id', 'tweet', 'tweet_created_at', 'location_profile'], dtype='object')
Number of tweets in ent: 47604376
Memory size of ent: 402267520


In [23]:
year='2019'

In [24]:
with open("./years/"+year+"_ent_year_ind.txt", "rb") as fp:   
    ent_year_ind=pickle.load(fp)

len(ent_year_ind)

2000000

In [25]:
with open("./years/ent_cleaning_classification.txt", "rb") as fp:  
    ent_tweets_rows=pickle.load(fp)

In [26]:
len(ent_tweets_rows)

47604376

In [16]:
with open("./years/"+year+"result_ent_bin_1.txt", "rb") as fp:  
    results_ent_2019_1=pickle.load(fp)

with open("./years/"+year+"result_ent_bin_2.txt", "rb") as fp:  
    results_ent_2019_2=pickle.load(fp)

with open("./years/"+year+"result_ent_bin_3.txt", "rb") as fp:  
    results_ent_2019_3=pickle.load(fp)

with open("./years/"+year+"result_ent_bin_4.txt", "rb") as fp:  
    results_ent_2019_4=pickle.load(fp)

with open("./years/"+year+"result_ent_bin_5.txt", "rb") as fp:  
    results_ent_2019_5=pickle.load(fp)

In [16]:
#with open("./years/"+year+"result_ent_2019_auto.txt", "rb") as fp:  
#    results_ent_2019_auto=pickle.load(fp)

In [17]:
#with open("./years/"+year+"_tweets_ent_scores_1.txt", "rb") as fp:  
#    dic_ent_tweets_scores=pickle.load(fp)

In [18]:
results_ent_2019_1[0:10]

array([[8.62196088e-03],
       [9.99997616e-01],
       [5.25146723e-04],
       [1.47143006e-03],
       [4.12635088e-01],
       [8.03491056e-01],
       [9.81172144e-01],
       [2.89643587e-10],
       [7.28281379e-01],
       [9.99671459e-01]])

In [19]:
vote_19=np.zeros(2000000)
label_19=np.zeros(2000000)
for i in range(0,2000000):
    
    temp=0
    
    if results_ent_2019_1[i] >0.5:
        res_1=1
    else:
        res_1=-1
    #print(results_ent_2019_1[i])    
        
    if results_ent_2019_2[i] >0.5:
        res_2=1
    else:
        res_2=-1      
    #print(results_ent_2019_2[i])    

    if results_ent_2019_3[i] >0.5:
        res_3=1
    else:
        res_3=-1
    #print(results_ent_2019_3[i])  
    
    if results_ent_2019_4[i] >0.5:
        res_4=1
    else:
        res_4=-1
    #print(results_ent_2019_4[i])    
        
    if results_ent_2019_5[i] >0.5:
        res_5=1
    else:
        res_5=-1      
    #print(results_ent_2019_5[i])    
    #print('+++++++')
         
        
    vote_19[i] = res_1 + res_2 + res_3 + res_4 + res_5  
    
    if vote_19[i] > 0 :
        label_19[i]=1
    else:
        label_19[i]=0
        
        
        

In [20]:
dic_vote_19=dict()

for i in vote_19:
    if i in dic_vote_19:
        dic_vote_19[i] +=1
    else:
        dic_vote_19[i] =1

dic_vote_19 = sorted(dic_vote_19.items(), key=operator.itemgetter(1), reverse=True) 
dic_vote_19

[(5.0, 505165),
 (-3.0, 436536),
 (-5.0, 434544),
 (-1.0, 236222),
 (3.0, 210342),
 (1.0, 177191)]

In [21]:
vote_19[0:10], label_19[0:10]

(array([-5.,  3., -3., -5., -1., -1.,  3., -5., -1.,  3.]),
 array([0., 1., 0., 0., 0., 0., 1., 0., 0., 1.]))

In [22]:
'''
vote1=np.zeros(2000000)
for i in range(0,2000000):
    
    temp=0
    
    if results_ent_2019_1[i] >0.5:
        res_1=1
    else:
        res_1=-1
    #print(results_ent_2019_1[i])    
        
    if results_ent_2019_2[i] >0.5:
        res_2=1
    else:
        res_2=-1      
    #print(results_ent_2019_2[i])    

    if results_ent_2019_3[i] >0.5:
        res_3=1
    else:
        res_3=-1
    #print(results_ent_2019_3[i])  
    
    if results_ent_2019_4[i] >0.5:
        res_4=1
    else:
        res_4=-1
    #print(results_ent_2019_4[i])    
        
    if results_ent_2019_5[i] >0.5:
        res_5=1
    else:
        res_5=-1      
    #print(results_ent_2019_5[i])    
    #print('+++++++')
         
    if results_ent_2019_auto[i] < 0.0052065444:
        res_auto=1
    else:
        res_auto=-1 
        
    vote1[i]=res_1 + res_2 +res_3 +res_4 +res_5  + res_auto    
'''
pass

In [23]:
'''
dic_vote1=dict()

for i in vote1:
    if i in dic_vote1:
        dic_vote1[i] +=1
    else:
        dic_vote1[i] =1

dic_vote1 = sorted(dic_vote1.items(), key=operator.itemgetter(1), reverse=True) 
dic_vote1
'''
pass

In [24]:
with open("./years/"+year+"result_ent_vote.txt", "wb") as fp:  
    pickle.dump(vote_19,fp)

with open("./years/"+year+"result_ent_label.txt", "wb") as fp:  
    pickle.dump(label_19,fp)

In [25]:
year='2017'

In [26]:
with open("./years/"+year+"_ent_year_ind.txt", "rb") as fp:   
    ent_year_ind=pickle.load(fp)

len(ent_year_ind)

2000000

In [27]:
#with open("./years/ent_cleaning_classification.txt", "wb") as fp:  
#    pickle.dump(ent_tweets_rows, fp , protocol=4)

In [28]:
with open("./years/"+year+"result_ent_bin_1.txt", "rb") as fp:  
    results_ent_2017_1=pickle.load(fp)

with open("./years/"+year+"result_ent_bin_2.txt", "rb") as fp:  
    results_ent_2017_2=pickle.load(fp)

with open("./years/"+year+"result_ent_bin_3.txt", "rb") as fp:  
    results_ent_2017_3=pickle.load(fp)

with open("./years/"+year+"result_ent_bin_4.txt", "rb") as fp:  
    results_ent_2017_4=pickle.load(fp)

with open("./years/"+year+"result_ent_bin_5.txt", "rb") as fp:  
    results_ent_2017_5=pickle.load(fp)

In [29]:
vote_17=np.zeros(2000000)
label_17=np.zeros(2000000)

for i in range(0,2000000):
    
    temp=0
    
    if results_ent_2017_1[i] >0.5:
        res_1=1
    else:
        res_1=-1
    #print(results_ent_2017_1[i])    
        
    if results_ent_2017_2[i] >0.5:
        res_2=1
    else:
        res_2=-1      
    #print(results_ent_2017_2[i])    

    if results_ent_2017_3[i] >0.5:
        res_3=1
    else:
        res_3=-1
    #print(results_ent_2017_3[i])  
    
    if results_ent_2017_4[i] >0.5:
        res_4=1
    else:
        res_4=-1
    #print(results_ent_2017_4[i])    
        
    if results_ent_2017_5[i] >0.5:
        res_5=1
    else:
        res_5=-1      
    #print(results_ent_2017_5[i])    
    #print('+++++++')
         
        
    vote_17[i] = res_1 + res_2 + res_3 + res_4 + res_5  
    
    if vote_17[i] > 0 :
        label_17[i]=1
    else:
        label_17[i]=0       

In [30]:
dic_vote_17=dict()

for i in vote_17:
    if i in dic_vote_17:
        dic_vote_17[i] +=1
    else:
        dic_vote_17[i] =1

dic_vote_17 = sorted(dic_vote_17.items(), key=operator.itemgetter(1), reverse=True) 
dic_vote_17

[(5.0, 667242),
 (-3.0, 397436),
 (-5.0, 326771),
 (3.0, 218694),
 (-1.0, 212580),
 (1.0, 177277)]

In [35]:
with open("./years/"+year+"result_ent_vote.txt", "wb") as fp:  
    pickle.dump(vote_17,fp)

with open("./years/"+year+"result_ent_label.txt", "wb") as fp:  
    pickle.dump(label_17,fp)